---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

[certificate](https://www.coursera.org/account/accomplishments/verify/5GA55KE5HZ9G)

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    resp = []
    for graph in P1_Graphs:
        degrees = graph.degree()
        degree_values = sorted(set(degrees.values()))
        #degress = graph.in_degree()
        degrees = [(list(degrees.values()).count(i),i) for i in degree_values]
        degrees = sorted(degrees, key = lambda x:x[1])
        degrees = [i for i,_ in degrees]
        avg_clustering = nx.average_clustering(graph)
        avg_path = nx.average_shortest_path_length(graph)
        if avg_clustering < 0.4:
            if degrees[0] == max(degrees):
                resp.append("PA")
            else:
                resp.append("SW_H")
        else:
            #print("SW_L")
            #print(degrees)
            resp.append("SW_L")
        
    return resp# Your Answer Here
#graph_identification()

In [4]:
list(reversed([5,4,3,2,1])) == sorted([5,4,3,2,1])

True

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
        
    # Your Code Here
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    
    df = pd.DataFrame(index= G.nodes())
    df["Department"] = pd.Series(nx.get_node_attributes(G, "Department"))
    df["ManagementSalary"] = pd.Series(nx.get_node_attributes(G, "ManagementSalary"))
    df["Degree"] = pd.Series(G.degree())
    df["Clustering"] = pd.Series(nx.clustering(G))
    df["degree_centrality"] = pd.Series(nx.degree_centrality(G))
    df["closeness_centrality"] = pd.Series(nx.closeness_centrality(G))
    
    df_test = df[df["ManagementSalary"].isnull()]
    df_train = df[df["ManagementSalary"] >= 0]

    X = df_train.drop(labels="ManagementSalary", axis=1)
    y = df_train["ManagementSalary"]
    
    X_test = df_test.drop(labels="ManagementSalary", axis=1)

    model = LogisticRegression(penalty = "l1", C=100).fit(X,y)
    preds = model.predict_proba(X_test)    
    
    return pd.Series(data=preds[:,1], index=list(df_test.index))# Your Answer Here
#salary_predictions()

In [7]:
#G.nodes(data=True)

In [8]:
"""df = pd.DataFrame(index= G.nodes())
df["Department"] = pd.Series(nx.get_node_attributes(G, "Department"))
df["ManagementSalary"] = pd.Series(nx.get_node_attributes(G, "ManagementSalary"))
df["Degree"] = pd.Series(G.degree())
df["Clustering"] = pd.Series(nx.clustering(G))
df["degree_centrality"] = pd.Series(nx.degree_centrality(G))
df["closeness_centrality"] = pd.Series(nx.closeness_centrality(G))
#df.head()
"""

'df = pd.DataFrame(index= G.nodes())\ndf["Department"] = pd.Series(nx.get_node_attributes(G, "Department"))\ndf["ManagementSalary"] = pd.Series(nx.get_node_attributes(G, "ManagementSalary"))\ndf["Degree"] = pd.Series(G.degree())\ndf["Clustering"] = pd.Series(nx.clustering(G))\ndf["degree_centrality"] = pd.Series(nx.degree_centrality(G))\ndf["closeness_centrality"] = pd.Series(nx.closeness_centrality(G))\n#df.head()\n'

In [9]:
"""df_test = df[df["ManagementSalary"].isnull()]
#df_train.head()
df_train = df[df["ManagementSalary"] >= 0]
#df_train.head()
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

X = df_train.drop(labels="ManagementSalary", axis=1)
y = df_train["ManagementSalary"]

clasifier = LogisticRegression()
parameters = {"penalty":["l1","l2"],"C":[0.001,0.01,0.1,10,50,100,1000]}
model = GridSearchCV(clasifier,param_grid=parameters,scoring="roc_auc")
model.fit(X,y)
"""

'df_test = df[df["ManagementSalary"].isnull()]\n#df_train.head()\ndf_train = df[df["ManagementSalary"] >= 0]\n#df_train.head()\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import roc_auc_score\n\nX = df_train.drop(labels="ManagementSalary", axis=1)\ny = df_train["ManagementSalary"]\n\nclasifier = LogisticRegression()\nparameters = {"penalty":["l1","l2"],"C":[0.001,0.01,0.1,10,50,100,1000]}\nmodel = GridSearchCV(clasifier,param_grid=parameters,scoring="roc_auc")\nmodel.fit(X,y)\n'

In [10]:
"""#model.cv_results_.keys()
#print(model.cv_results_["mean_test_score"])
#print(model.cv_results_["param_C"])
#print(model.cv_results_["param_penalty"])
#model.best_score_
#model.best_params_

#X_test = df_test.drop(labels="ManagementSalary", axis=1)

#model = LogisticRegression(penalty = "l1", C=100).fit(X,y)
#preds = model.predict(X_test)
#print(preds)
#df_test.index
preds = model.predict_proba(X_test)
print(preds[:,1])
#print(len(list(df_test.index)))"""

'#model.cv_results_.keys()\n#print(model.cv_results_["mean_test_score"])\n#print(model.cv_results_["param_C"])\n#print(model.cv_results_["param_penalty"])\n#model.best_score_\n#model.best_params_\n\n#X_test = df_test.drop(labels="ManagementSalary", axis=1)\n\n#model = LogisticRegression(penalty = "l1", C=100).fit(X,y)\n#preds = model.predict(X_test)\n#print(preds)\n#df_test.index\npreds = model.predict_proba(X_test)\nprint(preds[:,1])\n#print(len(list(df_test.index)))'

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [11]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [49]:
def new_connections_predictions():
    
    # Your Code Here
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    
    df = pd.DataFrame(index=future_connections.index)
    df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['jaccard_coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['resource_allocation_index'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    df['adamic_adar_index'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    df['Common Neighbors'] = df.index.map(lambda city: len(list(nx.common_neighbors(G, city[0], city[1]))))
    
    X_test = df[future_connections["Future Connection"].isnull()]
    X_train = df[future_connections["Future Connection"] >= 0]
    

    y_train = future_connections[future_connections["Future Connection"] >= 0]
    y_train = np.array(y_train).reshape(len(y_train),)

    model = LogisticRegression(penalty="l1", C=1000).fit(X_train,y_train)
    preds = model.predict_proba(X_test)
    
    return pd.Series(data=preds[:,1], index=list(X_test.index))# Your Answer Here# Your Answer Here
#new_connections_predictions()

(107, 348)    0.028261
(542, 751)    0.011175
(20, 426)     0.563165
(50, 989)     0.011258
(942, 986)    0.011347
(324, 857)    0.011246
(13, 710)     0.189277
(19, 271)     0.121007
(319, 878)    0.011287
(659, 707)    0.011184
(49, 843)     0.011313
(208, 893)    0.011204
(377, 469)    0.008105
(405, 999)    0.019834
(129, 740)    0.016527
(292, 618)    0.029697
(239, 689)    0.011292
(359, 373)    0.009301
(53, 523)     0.038893
(276, 984)    0.011308
(202, 997)    0.011326
(604, 619)    0.165145
(270, 911)    0.011291
(261, 481)    0.073264
(200, 450)    0.903692
(213, 634)    0.011149
(644, 735)    0.102498
(346, 553)    0.010944
(521, 738)    0.010552
(422, 953)    0.020329
                ...   
(672, 848)    0.011291
(28, 127)     0.938444
(202, 661)    0.010909
(54, 195)     0.999991
(295, 864)    0.011234
(814, 936)    0.011162
(839, 874)    0.011347
(139, 843)    0.011241
(461, 544)    0.010280
(68, 487)     0.010233
(622, 932)    0.011214
(504, 936)    0.017414
(479, 528) 

[(0, 1, {}),
 (0, 17, {}),
 (0, 316, {}),
 (0, 146, {}),
 (0, 581, {}),
 (0, 268, {}),
 (0, 221, {}),
 (0, 218, {}),
 (0, 18, {}),
 (0, 734, {}),
 (0, 178, {}),
 (0, 380, {}),
 (0, 0, {}),
 (0, 459, {}),
 (0, 215, {}),
 (0, 250, {}),
 (0, 148, {}),
 (0, 73, {}),
 (0, 74, {}),
 (0, 248, {}),
 (0, 498, {}),
 (0, 226, {}),
 (0, 101, {}),
 (0, 377, {}),
 (0, 177, {}),
 (0, 103, {}),
 (0, 560, {}),
 (0, 309, {}),
 (0, 88, {}),
 (0, 5, {}),
 (0, 297, {}),
 (0, 313, {}),
 (0, 223, {}),
 (0, 238, {}),
 (0, 368, {}),
 (0, 266, {}),
 (0, 222, {}),
 (0, 283, {}),
 (0, 6, {}),
 (0, 64, {}),
 (0, 65, {}),
 (0, 166, {}),
 (0, 120, {}),
 (1, 74, {}),
 (1, 17, {}),
 (1, 316, {}),
 (1, 1, {}),
 (1, 268, {}),
 (1, 495, {}),
 (1, 377, {}),
 (1, 549, {}),
 (1, 310, {}),
 (1, 218, {}),
 (1, 215, {}),
 (1, 147, {}),
 (1, 106, {}),
 (1, 21, {}),
 (1, 225, {}),
 (1, 82, {}),
 (1, 254, {}),
 (1, 155, {}),
 (1, 146, {}),
 (1, 85, {}),
 (1, 284, {}),
 (1, 189, {}),
 (1, 250, {}),
 (1, 726, {}),
 (1, 548, {}),
 (

In [20]:
"""df = pd.DataFrame(index=future_connections.index)
df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
df['jaccard_coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
df['resource_allocation_index'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
df['adamic_adar_index'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
df['Common Neighbors'] = df.index.map(lambda city: len(list(nx.common_neighbors(G, city[0], city[1]))))
"""

In [44]:
"""X_test = df[future_connections["Future Connection"].isnull()]
#df_train.head()
X_train = df[future_connections["Future Connection"] >= 0]
#df_train.head()
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

#X = df_train.drop(labels="ManagementSalary", axis=1)
y_train = future_connections[future_connections["Future Connection"] >= 0]
y_train = np.array(y_train).reshape(len(y_train),)

clasifier = LogisticRegression()
parameters = {"penalty":["l1","l2"],"C":[0.001,0.01,0.1,10,50,100,1000]}
model = GridSearchCV(clasifier,param_grid=parameters,scoring="roc_auc")
model.fit(X_train,y_train)"""

#print(len(X_train))
#print(np.array(y_train).reshape(len(y_train),))

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 10, 50, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [47]:
#model.best_score_
#model.best_params_

{'C': 1000, 'penalty': 'l1'}